In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
train_data = pd.read_csv("../data/processed/train_data_baseline.csv")
test_data = pd.read_csv("../data/processed/test_data_baseline.csv")

In [3]:
class filter_add_attributes(BaseEstimator, TransformerMixin):
    '''Custom transformer based on Sklearn's classes.
    Takes in dataframe (train or test) and adds new features and returns
    a filtered version of the original train/test datasets.'''
    def fit(self, X, y=None):
        return self.fit_transform(X)
    def transform(self, X, y=None):
        return self.fit_transform(X)
    def fit_transform(self, X, y=None):
        '''Calculates and adds comment body length and account activity (based on frequency of comment author)
        as features. Returns a new dataframe with the added columns.
        
        Arguments:
            X: dataframe to transform (train or test)
        Returns:
            Transformed and filtered version of train or test set
        '''
        data = X.copy()
        data["body_len"] = data.comment_body.apply(lambda x: len(x))
        data["acc_activity"] = data.author_ids.map(data.author_ids.value_counts())
        data["is_premium"] = data.is_premium.astype(int)
        result = data.filter(items=["ups", "comment_karma", "link_karma", "is_premium", 
                                  "comment_age_days", "acc_age_days", "body_len", "acc_activity"], axis=1)
        return result

In [4]:
pipeline = Pipeline([
        ('filter_add', filter_add_attributes()),
        ('scaler', StandardScaler()),
    ])

X_train = pipeline.fit_transform(train_data)
X_test = pipeline.transform(test_data)
y_train = train_data["gildings"].to_list()
y_test = test_data["gildings"].to_list()
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

We will defined appropriate functions for training and returning cross validation results for our logistic regression model.

Due to the high class imbalance, F1 score is a much better metric to use than just accuracy (since 99% of the data belongs to class 0). We will also have ROC-AUC for comparison.

We fetch the true positives, false positives and false negatives to calculate the f1 score across all folds rather than using the builtin functionality. This is because the averaged f1 score returned by Sklearn is slighly biased for imbalanced class problems (for cross validation). This doesn't matter when evaluating the test set.

Note that we do include the default implementation of f1 score metric in scoring; this is for later use (to tune GridSearch).

Reference: https://www.hpl.hp.com/techreports/2009/HPL-2009-359.pdf

In [5]:
# Defining the functions to get true positives...false negatives for Sklearn's make_scorer function
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
scoring = {'tp': make_scorer(tp), 'fp': make_scorer(fp), 
           'fn': make_scorer(fn), 'roc_auc': make_scorer(roc_auc_score),
          'f1': make_scorer(f1_score)}

def train(clf, X, y):
    '''Takes in train and train datasets along a model to train and evaluate. Returns f1 and roc-auc scores.
    
    Arguments:
        clf: classifier to use
        X: Dataframe corresponding to training set
        y: Targets corresponding to the training set
    Returns:
        Cross_validate output.
    '''
    rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=42)
    scores = cross_validate(clf, X, y, cv=rskf, scoring=scoring)
    return scores

def calc_metrics(scores):
    '''Takes in cross_validate outputs, gets arrays of true positives, false negatives and false positives 
    (corresponding to class 1; gilded) across k folds and calculates the f1 score. Calculates the mean of
    roc_auc scores from k folds. Returns both f1 and roc_auc scores.
    
    Arguments:
        Cross_validate outputs
    Returns:
        F1 score, calculated using the totals, as well as ROC_AUC score from scores.
        ROC_AUC score, for the best parameters
    '''
    tps, fns, fps, roc = scores['test_tp'], scores['test_fn'], scores['test_fp'], scores['test_roc_auc']
    tp = np.sum(tps)
    fn = np.sum(fns)
    fp = np.sum(fps)
    f1 = (2*tp)/(2*tp+fn+fp)
    return f1, np.mean(scores['test_roc_auc'])

Let's establish a baseline model that simply predicts the minority class:

In [6]:
clf = DummyClassifier(strategy="constant", constant=1)
scores = train(clf, X_train, y_train)
f1, roc = calc_metrics(scores)
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")

F1 score is: 0.0038792557287250116
ROC-AUC is: 0.5


A good model is one that can perform better than the baseline, in terms of F1 Score. Anything below is worse than a model that simply predicts minority class.

Note that 0.5 ROC-AUC score indicates that it's a random classifier.

In [7]:
scores = train(LogisticRegression(), X_train, y_train)
f1, roc = calc_metrics(scores)
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")

F1 score is: 0.14389101745423585
ROC-AUC is: 0.5423767409646068


Looks like it's slightly better than a random classifier; this means that our model is learning some relationships for the underlying data, albeit small.

The low score is to expected, especially given the class imbalance. Let's try using the class weight functionality that assigns weights to each class based on their frequency.

In [20]:
clf = LogisticRegression(class_weight='balanced')
scores = train(clf, X_train, y_train)
f1, roc = calc_metrics(scores)
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")

F1 score is: 0.07312321528648821
ROC-AUC is: 0.8275665348137995


In [21]:
print(scores)

{'fit_time': array([1.31030202, 1.16940689, 1.14577937, 1.09115148, 1.13671255,
       1.17197299, 1.08603263, 1.18000984, 1.13265562, 1.13404036,
       1.18123364, 1.14052343, 1.13511944, 1.04346681, 1.08995152,
       1.24200678, 1.08979511, 1.1988821 , 1.03887463, 1.15954828]), 'score_time': array([0.19030547, 0.18034387, 0.18628407, 0.18020415, 0.18183947,
       0.18124151, 0.17997718, 0.18089461, 0.18050003, 0.18194246,
       0.18317223, 0.18202949, 0.18137407, 0.18184066, 0.1816113 ,
       0.18208528, 0.18212032, 0.17977977, 0.18108582, 0.18056846]), 'test_tp': array([71, 72, 71, 66, 63, 70, 64, 72, 75, 61, 65, 79, 69, 76, 63, 66, 72,
       69, 65, 61]), 'test_fp': array([1771, 1869, 1786, 1420, 1483, 1729, 1724, 1771, 2053, 1462, 1779,
       1964, 1688, 2005, 1684, 1628, 1636, 1844, 1367, 1448]), 'test_fn': array([28, 27, 28, 33, 36, 30, 36, 28, 25, 39, 34, 20, 30, 23, 36, 34, 28,
       31, 35, 39]), 'test_roc_auc': array([0.84125709, 0.84534869, 0.84111032, 0.81943901, 0

In [9]:
clf = LogisticRegression(class_weight='balanced')
scores = train(clf, X_train, y_train)
f1, roc = calc_metrics(scores)
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")

F1 score is: 0.07312321528648821
ROC-AUC is: 0.8275665348137995


Looks like the balanced class weight performs worse in terms of f1 score (probably because it results in a lot more false positives).

Let's define functions to test different parameters using GridSearchCV. Note that we are also defining a custom function to calculate the f1 score (as earlier).

In [10]:
def train_grid(X_train, y_train, parameters):
    '''Function to train and find the best hyperparameters for Logistic Regression.
    Uses GridSearch and crossvalidation for tuning.
    
    Arguments:
        X_train: Training set (pandas Dataframe)
        y_train: Targets corresponding to the training set
        parameters: Parameters to test/run GridSearch with.
        
    Returns:
        Classifier, which is used to parse the outputs/metrics.
    '''
    model = LogisticRegression(random_state=42)
    rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=42)
    clf = GridSearchCV(model, parameters, scoring=scoring, cv=rskf, refit='f1')
    clf.fit(X_train, y_train)
    return clf

def calc_metrics_grid(clf):
    '''Arguments:
        Classifier which is used to parse all the relevant outputs
    Returns: 
        F1 score, calculated with True Positives, False Positives and False Negatives across k (k=20) folds
    '''
    ind = clf.best_index_
    tp_list = ['split' + str(i) + '_test_tp' for i in range(20)]
    fp_list = ['split' + str(i) + '_test_fp' for i in range(20)]
    fn_list = ['split' + str(i) + '_test_fn' for i in range(20)]
    tps, fps, fns = 0, 0, 0
    for tp in tp_list:
        tps+= clf.cv_results_[tp][ind]
    for fp in fp_list:
        fps+= clf.cv_results_[fp][ind]
    for fn in fn_list:
        fns+= clf.cv_results_[fn][ind]
    f1 = (2*tps)/(2*fps+fns+fps)
    return f1, clf.cv_results_['mean_test_roc_auc'][ind]

In [11]:
parameters = {'class_weight':[{0:1,1:1}, {0:1,1:10}, {0:1,1:100}, {0:1,1:1000}, {0:10,1:1}]}
clf = train_grid(X_train, y_train, parameters)
best = clf.best_params_
f1, roc = calc_metrics_grid(clf)
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")
print(f"Best class weights: {best}")

F1 score is: 0.2146809295462929
ROC-AUC is: 0.6456231724605754
Best class weights: {'class_weight': {0: 1, 1: 10}}


We are making progress, but can we do even better?

Adjusting the weights were not enough, we will have to try different sampling techniques. Imbalanced-learn library will come in handy here.

We will start with RandomOverSampler to duplicates records from the minority class. We will use a sampling ratio of 0.1 (i.e. ~10% increase in gilded class).

Read more: https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html#a-practical-guide

In [12]:
#Using RandomOverSampler to duplicate records belonging to class 1 (gilded)

random_sampler = RandomOverSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = random_sampler.fit_resample(X_train, y_train)
assert len(X_resampled) == len(y_resampled)
clf = LogisticRegression(class_weight={0: 1, 1: 10})
scores = train(clf, X_resampled, y_resampled)
f1, roc = calc_metrics(scores)
print("Random Over Sampling:")
print(f"F1 score is: {f1}")
print(f"ROC-AUC is: {roc}")

Random Over Sampling:
F1 score is: 0.6781609747429389
ROC-AUC is: 0.828094671658085


We can also generate new samples with SMOTE and ADASYN based on existing samples. We will keep the sampling ratio the same for comparison.

In [13]:
#Using SMOTE and ADASYN to generate samples in gilded class
smote = SMOTE(sampling_strategy=0.1, random_state=42)
ada = ADASYN(sampling_strategy=0.1, random_state=42)
models = [smote, ada]
model_names = ["SMOTE", "ADASYN"]

for i in range(len(models)):
    X_resampled, y_resampled = models[i].fit_resample(X_train, y_train)
    assert len(X_resampled) == len(y_resampled)
    clf = LogisticRegression(class_weight=[{0: 1, 1: 10}])
    scores = train(clf, X_resampled, y_resampled)
    f1, roc = calc_metrics(scores)
    print(f"{model_names[i]} :")
    print(f"F1 score is: {f1}")
    print(f"ROC-AUC is: {roc}")
    print("\n")

SMOTE :
F1 score is: 0.6098306864417903
ROC-AUC is: 0.7254990149796094


ADASYN :
F1 score is: 0.5570636250902797
ROC-AUC is: 0.6984619951518117




Imbalanced learn also recommends combining oversampling with undersampling the majority class.

Ref: https://imbalanced-learn.readthedocs.io/en/stable/auto_examples/combine/plot_comparison_combine.html

SMOTE can generate noisy samples (ex: when classes cannot be well separated), undersampling allows to clean the noisy data.

In [14]:
smote_tomek = SMOTETomek(sampling_strategy=0.1, random_state=42)
smote_enn = SMOTEENN(sampling_strategy=0.1, random_state=42)

models = [smote_tomek, smote_enn]
model_names = ["SMOTE TOMEK", "SMOTE ENN"]

for i in range(len(models)):
    X_resampled, y_resampled = models[i].fit_resample(X_train, y_train)
    assert len(X_resampled) == len(y_resampled)
    clf = LogisticRegression(class_weight=[{0: 1, 1: 10}])
    scores = train(clf, X_resampled, y_resampled)
    f1, roc = calc_metrics(scores)
    print(f"{model_names[i]} :")
    print(f"F1 score is: {f1}")
    print(f"ROC-AUC is: {roc}")
    print("\n")

SMOTE TOMEK :
F1 score is: 0.6117789251041132
ROC-AUC is: 0.7265310228680512


SMOTE ENN :
F1 score is: 0.7003509844764216
ROC-AUC is: 0.7759026910883595




SMOTE, SMOTEENN and RandomOverSampler produces the best results so far. Let's evaluate those them on our test set.

In [17]:
random_sampler = RandomOverSampler(sampling_strategy=0.1, random_state=42)
smote = SMOTE(sampling_strategy=0.1, random_state=42)
smote_enn = SMOTEENN(sampling_strategy=0.1, random_state=42)
models = [random_sampler, smote, smote_enn]
model_names = ["Random Oversampling", "SMOTE", "SMOTE ENN"]

for i in range(len(models)):
    parameters = {'class_weight':[{0:1,1:10}]}
    X_resampled, y_resampled = models[i].fit_resample(X_train, y_train)
    grid_res = train_grid(X_resampled, y_resampled, parameters)
    y_preds = grid_res.predict(X_test)
    print(f"{model_names[i]} on test set:")
    print(f"F1 score: {f1_score(y_test, y_preds)}")
    print(f"ROC_AUC score: {roc_auc_score(y_test, y_preds)}")
    print(f"Balanced accuracy score: {balanced_accuracy_score(y_test, y_preds)}")
    print("\n")

Random Oversampling on test set:
F1 score: 0.0638904734740445
ROC_AUC score: 0.8183981729232407
Balanced accuracy score: 0.8183981729232408


SMOTE on test set:
F1 score: 0.060296191819464044
ROC_AUC score: 0.8228175600742684
Balanced accuracy score: 0.8228175600742684


SMOTE ENN on test set:
F1 score: 0.08897775556110973
ROC_AUC score: 0.8434413510604898
Balanced accuracy score: 0.8434413510604898




Logistic regression predicts the class probabilities for each sample and decides class based on a threshold (default: 0.5). We can also check if a different threshold value produces better results.

Ref: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

Let's define the relevant functions first.

In [18]:
def train_eval_probs(X, y, X_test):
    '''Takes in train and train datasets along a model to train and evaluate. Returns f1 and roc-auc scores.
    Arguments:
        X: Training dataset
        y: Targets corresponding to the training set
        X_test: Test dataset for calculating class probabilities
    Returns:
        Class probabilities associated with each sample (only returns probabilities for class 1)'''
    rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=42)
    model = LogisticRegressionCV(cv=rskf, class_weight=[{0: 1, 1: 10}])
    model.fit(X, y)
    return model.predict_proba(X_test)[:,1]

def convert_probs(probs, threshold):
    return (probs >= threshold).astype('int')

In [19]:
random_sampler = RandomOverSampler(sampling_strategy=0.1, random_state=42)
smote = SMOTE(sampling_strategy=0.1, random_state=42)
smote_enn = SMOTEENN(sampling_strategy=0.1, random_state=42)
models = [random_sampler, smote, smote_enn]
model_names = ["Random Oversampling", "SMOTE", "SMOTE ENN"]
thresholds = np.arange(0, 1, 0.001)

for i in range(len(models)):
    parameters = {'class_weight':[{0:1,1:10}]}
    X_resampled, y_resampled = models[i].fit_resample(X_train, y_train)
    probs = train_eval_probs(X_resampled, y_resampled, X_test)
    f1_scores = [f1_score(y_test, convert_probs(probs, t)) for t in thresholds]
    auc_scores = [roc_auc_score(y_test, convert_probs(probs, t)) for t in thresholds]
    ind1 = np.argmax(f1_scores)
    ind2 = np.argmax(auc_scores)
    print(f"\n{model_names[i]} on test set:")
    print("Maxiziming F1 Score:")
    print(f"Threshold: {thresholds[ind1]}, F1 Score: {f1_scores[ind1]}, ROC AUC: {auc_scores[ind1]}")
    print("Maxiziming ROC-AUC Score:")
    print(f"Threshold: {thresholds[ind2]}, F1 Score: {f1_scores[ind2]}, ROC AUC: {auc_scores[ind2]}")


Random Oversampling on test set:
Maxiziming F1 Score:
Threshold: 0.9470000000000001, F1 Score: 0.3143350604490501, ROC AUC: 0.681795495628806
Maxiziming ROC-AUC Score:
Threshold: 0.114, F1 Score: 0.06988058381247236, ROC AUC: 0.8011632750856418

SMOTE on test set:
Maxiziming F1 Score:
Threshold: 0.9470000000000001, F1 Score: 0.31762652705061084, ROC AUC: 0.6818189791785794
Maxiziming ROC-AUC Score:
Threshold: 0.115, F1 Score: 0.07159142726858185, ROC AUC: 0.7996836228270289

SMOTE ENN on test set:
Maxiziming F1 Score:
Threshold: 0.998, F1 Score: 0.3019431988041854, ROC AUC: 0.7015627029169681
Maxiziming ROC-AUC Score:
Threshold: 0.107, F1 Score: 0.08415217939027463, ROC AUC: 0.8214351900710419


Overall, our results are better than our baseline model, but not ideal. Perhaps, we can achieve better results with a more complex (non-linear) model. Let's try SVM next.